In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from functools import partial

# --- Closeness Metric Functions ---

def rbf_kernel(squared_distances, t=1.0):
    """
    Implements the default RBF (Gaussian) kernel for sample similarity S_ij.
    S_ij = exp(-||x_i - x_j||^2 / t)
    """
    return np.exp(-squared_distances / t)

# --- Laplacian Score Calculation Function ---

def calculate_laplacian_score(X, k, closeness_metric_func, **metric_params):
    """
    Computes the Laplacian Score (Lr) for each feature using a user-defined
    closeness metric function to build the similarity matrix S.
    """
    m, n = X.shape

    # --- 1. Construct the nearest neighbor graph G and weight matrix S ---

    # 1.1 Calculate all pairwise squared Euclidean distances (R_sq)
    sum_sq_X = np.sum(X**2, axis=1, keepdims=True)
    R_sq = sum_sq_X + sum_sq_X.T - 2 * np.dot(X, X.T)
    R_sq[R_sq < 0] = 0

    # 1.2 Find k-NN indices (required for graph structure)
    nbrs = NearestNeighbors(n_neighbors=k + 1).fit(X)
    distances, indices = nbrs.kneighbors(X)

    # 1.3 Initialize and populate S symmetrically using the custom metric
    S = np.zeros((m, m))
    for i in range(m):
        neighbor_indices = indices[i, 1:]

        weights = closeness_metric_func(R_sq[i, neighbor_indices], **metric_params)

        S[i, neighbor_indices] = weights
        S[neighbor_indices, i] = weights

    # --- 2. D and L (Vectorized) ---
    D_vector = np.sum(S, axis=1)
    D = np.diag(D_vector)
    L = D - S

    d_sum = np.sum(D_vector)
    if d_sum == 0:
         return np.zeros(n)

    # --- 3. Weighted Mean Removal (tilde(F)) (Vectorized) ---
    M = (X.T @ D_vector) / d_sum
    tilde_F = X - np.outer(np.ones(m), M)

    # --- 4. Laplacian Score Lr (Fully Vectorized Matrix Products) ---

    # Numerator Matrix: tilde(F)^T L tilde(F). Diagonal contains all numerators.
    Numerator_Matrix = tilde_F.T @ L @ tilde_F
    Numerator_vector = np.diag(Numerator_Matrix)

    # Denominator Matrix: tilde(F)^T D tilde(F). Diagonal contains all denominators (Weighted Variance).
    Denominator_Matrix = tilde_F.T @ D @ tilde_F
    Denominator_vector = np.diag(Denominator_Matrix)

    # Final Lr: Element-wise division with safety check
    L_r_scores = np.divide(Numerator_vector, Denominator_vector,
                           out=np.zeros_like(Numerator_vector, dtype=float),
                           where=Denominator_vector != 0)

    return L_r_scores

In [ ]:
# Placeholder data: 10 samples, 4 features
X_example = np.array([
    [5.1, 3.5, 1.4, 0.2],
    [4.9, 3.0, 1.4, 0.2],
    [4.7, 3.2, 1.3, 0.2],
    [4.6, 3.1, 1.5, 0.2],
    [5.0, 3.6, 1.4, 0.2],
    [7.0, 3.2, 4.7, 1.4],
    [6.4, 3.2, 4.5, 1.5],
    [6.9, 3.1, 4.9, 1.5],
    [5.5, 2.3, 4.0, 1.3],
    [6.5, 2.8, 4.6, 1.5]
])

k_neighbors = 5

# Call with RBF kernel and parameter t=1.0
scores_rbf = calculate_laplacian_score(
    X_example,
    k_neighbors,
    closeness_metric_func=rbf_kernel,
    t=1.0
)

print(f"Scores (RBF Kernel): {scores_rbf}")
# Scores (RBF Kernel): [0.08395097 0.89473051 0.00748533 0.00391253]

Scores (RBF Kernel): [0.08395097 0.89473051 0.00748533 0.00391253]


In [ ]:
def fisher_supervised_weight_matrix(labels):
    """
    Computes the supervised weight matrix S based on class labels, as defined
    in the Fisher Score section (Eq. 5).

    This function generates the full S matrix, bypassing k-NN and R_sq calculation.

    Args:
        labels (numpy.ndarray): A 1D array of class labels for the data samples (y_i).

    Returns:
        numpy.ndarray: The weight matrix S, shape (m, m).
    """
    m = len(labels)
    S = np.zeros((m, m))

    unique_labels, counts = np.unique(labels, return_counts=True)
    label_to_n = dict(zip(unique_labels, counts))

    # Fully vectorized calculation of S
    for label in unique_labels:
        # Find indices of all samples belonging to this class 'l'
        indices = np.where(labels == label)[0]
        n_l = label_to_n[label]
        weight = 1.0 / n_l

        # S_ij = 1/n_l for all pairs (i, j) within the same class 'l'
        # This creates an n_l x n_l block matrix (S_l) where every entry is 1/n_l
        S[np.ix_(indices, indices)] = weight

    return S

In [ ]:
def calculate_laplacian_score_supervised(X, labels):
    """
    Computes the Laplacian Score (Lr) using the supervised S matrix
    defined in the paper's Fisher Score section (Eq. 5).

    Args:
        X (numpy.ndarray): The data matrix, shape (m_samples, n_features).
        labels (numpy.ndarray): A 1D array of class labels (y_i).

    Returns:
        numpy.ndarray: A vector of Laplacian Scores, Lr, for each feature.
    """
    m, n = X.shape

    # --- 1. Compute S directly from labels ---
    S = fisher_supervised_weight_matrix(labels)

    # --- 2. D and L (Vectorized) ---
    D_vector = np.sum(S, axis=1) # Note: For this specific S, D_vector will be all ones if labels are valid
    D = np.diag(D_vector)
    L = D - S

    d_sum = np.sum(D_vector)
    if d_sum == 0:
         return np.zeros(n)

    # --- 3. Weighted Mean Removal (tilde(F)) (Vectorized) ---
    M = (X.T @ D_vector) / d_sum
    tilde_F = X - np.outer(np.ones(m), M)

    # --- 4. Laplacian Score Lr (Fully Vectorized Matrix Products) ---
    Numerator_Matrix = tilde_F.T @ L @ tilde_F
    Numerator_vector = np.diag(Numerator_Matrix)

    Denominator_Matrix = tilde_F.T @ D @ tilde_F
    Denominator_vector = np.diag(Denominator_Matrix)

    L_r_scores = np.divide(Numerator_vector, Denominator_vector,
                           out=np.zeros_like(Numerator_vector, dtype=float),
                           where=Denominator_vector != 0)

    return L_r_scores

Comparison Example

In [ ]:
# Data matrix (6 samples, 2 features)
X_comp = np.array([
    [1.0, 9.0], [1.5, 8.5], [1.2, 9.5],
    [8.0, 1.0], [8.5, 1.5], [8.2, 0.8]
])

# Class labels (n_0 = 3, n_1 = 3)
labels_comp = np.array([0, 0, 0, 1, 1, 1])